In [11]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from scipy.spatial import cKDTree

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
#pd.set_option('max_colwidth', 50)

In [13]:
df_path = 'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_with_grids_50km.csv'
df = pd.read_csv(df_path, index_col=0)

C:\Users\simon\AppData\Local\Temp\ipykernel_8484\1772750792.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, index_col=0)


In [14]:
lucas_path = 'D:\\Simon\\Documents\\GP\\data\\util_files\\LUCAS\\LUCA_PT_2009.shp'
lucas_df = gpd.read_file(lucas_path)
lucas_df.rename(columns={'LAT': 'coord_lat'}, inplace=True)
lucas_df.rename(columns={'LON': 'coord_lon'}, inplace=True)

In [15]:
lucas_df.head(2)

,LUCA_PT_ID,SURV_DATE,LUCA_OBSR_,LUCA_OBSR1,GPS_PROJ,GPS_PREC,LAT_GPS,LON_GPS,OBSR_DIST,LUCA_DIRE_,LUCA_LCOV_,LUCA_LCOV1,LUCA_RADI_,LUCA_LUSE_,LUCA_LUSE1,LUCA_SPCS_,LUCA_PERC_,LUCA_SPCS1,LUCA_PERC1,LUCA_AREA_,LUCA_TREE_,LUCA_FEAT_,LAND_GRAZ_,LUCA_WTMN_,LUCA_IRRI_,LUCA_IRRI1,LUCA_IRR_1,LAT_LAEA,LON_LAEA,LUCA_STRT_,LUCA_STRT1,NUTS_CODE,STAT_LEVL_,PHOT_PT_FL,PHOT_CROP_,PHOT_N_FLA,PHOT_E_FLA,PHOT_S_FLA,PHOT_W_FLA,PHOT_IRRI_,PHOT_TRSC_,PHOT_SOIL_,PHOT_CONF_,TRANS,ELEV,AREA,EAST,NORT,coord_lon,coord_lat,geometry
0,46702602,2-Jan-09,2.0,3.0,X,8888.0,89.0,89.0,88888.0,1.0,E20,8,2.0,U120,8,NaN,3.0,NaN,0.0,3.0,0.0,0.0,3.0,8.0,0.0,0.0,0.0,4670000,2602000,4,0,AT213,3,8.0,8.0,2,2,2,2.0,8,8,8.0,0.0,"PI(,E20,C20,)PI, , , , , , , , , , , , , , , ,...",945,3376.87552,4670000,2602000,14.539452,46.435266,POINT (14.53945 46.43527)
1,46422610,06JUL2009,1.0,1.0,WGS84,9.0,47.0,14.0,1.0,1.0,C10,8,2.0,U120,8,C16,5.0,NaN,0.0,3.0,2.0,2.0,2.0,8.0,0.0,0.0,0.0,4642000,2610000,4,0,AT211,3,1.0,1.0,1,1,1,1.0,8,1,8.0,2.0,"C10,C30,31,C10,32,C10, , , , , , , , , , , , ,...",680,2026.70874,4642000,2610000,14.181589,46.521961,POINT (14.18159 46.52196)


In [16]:
df.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,-1.051010,0.0,0,37718.0,de,50kmE4200N3300
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,0.760781,2.0,0,37803.0,de,50kmE4250N3300
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,270.831300,2.0,0,39627.0,de,50kmE4450N3050
3,29666414,8.0,Haubentaucher,2018-01-01,NaN,51.076006,11.038316,place,158.941010,8.0,0,38301.0,de,50kmE4350N3100
4,29656211,8.0,Haubentaucher,2018-01-01,NaN,51.389380,7.067282,place,52.362160,10.0,0,108167.0,de,50kmE4100N3100


In [17]:
df['geometry'] = gpd.points_from_xy(df['coord_lat'], df['coord_lon'])
lucas_df['geometry'] = gpd.points_from_xy(lucas_df['coord_lat'], lucas_df['coord_lon'])

# Create a spatial index (cKDTree) for the points in lucas_df
tree = cKDTree(lucas_df[['coord_lon', 'coord_lat']])

# Find the nearest point in lucas_df for each point in df
df['nearest_idx'] = df.apply(lambda row: tree.query([row['coord_lon'], row['coord_lat']])[1], axis=1)

# Merge the nearest points' features from lucas_df into df
merged_df = pd.merge(df, lucas_df[['LUCA_LCOV_', 'LUCA_LUSE_', 'coord_lat', 'coord_lon']], left_on='nearest_idx', right_index=True, suffixes=('_df', '_lucas_df'))

merged_df.drop(columns=['geometry','nearest_idx'], axis=1, inplace=True)

In [18]:
merged_df

,id_sighting,id_species,name_species,date,timing,coord_lat_df,coord_lon_df,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id,LUCA_LCOV_,LUCA_LUSE_,coord_lat_lucas_df,coord_lon_lucas_df
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,-1.051010,0.0,0,37718.0,de,50kmE4200N3300,E20,U111,53.142976,8.669968
5025,63901456,52.0,Singschwan,2022-01-01,NaN,53.149150,8.694349,place,-1.500000,5.0,0,37718.0,de,50kmE4200N3300,E20,U111,53.142976,8.669968
5026,63901443,8.0,Haubentaucher,2022-01-01,NaN,53.149150,8.694349,place,-1.500000,13.0,0,37718.0,de,50kmE4200N3300,E20,U111,53.142976,8.669968
23608,39257987,399.0,Steinschmätzer,2019-05-01,NaN,53.125139,8.655436,precise,1.139247,8.0,0,39590.0,de,50kmE4200N3300,E20,U111,53.142976,8.669968
34416,67928406,8.0,Haubentaucher,2022-05-01,04:20,53.124069,8.659462,precise,-2.000000,3.0,0,39908.0,de,50kmE4200N3300,E20,U111,53.142976,8.669968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620099,25674059,160.0,Gänsegeier,2022-05-22,NaN,46.031290,8.672069,square,703.056760,20.0,0,7282.0,ch,50kmE4200N2500,C10,U120,46.018660,8.671471
2638893,26048130,399.0,Steinschmätzer,2022-07-17,NaN,46.142045,6.574331,precise,1663.538700,2.0,19,518.0,ch,50kmE4050N2550,F00,U400,46.097270,6.549931
2642881,26027726,319.0,Sperlingskauz,2022-07-10,NaN,46.137527,6.568191,precise,1736.683000,1.0,2,518.0,ch,50kmE4050N2550,F00,U400,46.097270,6.549931
2642883,26027763,463.0,Wiesenpieper,2022-07-10,NaN,46.137527,6.568191,precise,1736.683000,3.0,2,518.0,ch,50kmE4050N2550,F00,U400,46.097270,6.549931
